# Neuronal networks
Proyecto 2

In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import pandas as pd
import seaborn as sb
from sklearn import datasets
from sklearn.model_selection import train_test_split
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.stats.diagnostic as diag
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import learning_curve

from sklearn.compose import make_column_selector as selector #Para seleccionar de forma automática las variables numéricas y categóricas
from sklearn.preprocessing import OneHotEncoder #Para codificar las variables categóricas usando dummies
from sklearn.preprocessing import StandardScaler #Para normalizar las variables numéricas
from sklearn.compose import ColumnTransformer #Modifica las columnas usando los preprocesadores
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline #Planifica una secuencia de procesos
from sklearn import set_config #Para mostrar graficamente el pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.model_selection import learning_curve

set_config(display='diagram')
#Metrics
from sklearn.metrics import make_scorer, accuracy_score,precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score ,precision_score,recall_score,f1_score

In [20]:
songs = pd.read_csv('songs.csv')
songs.describe()

,duration_ms,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,228748.124500,2009.49400,59.872500,0.667438,0.720366,5.378000,-5.512434,0.553500,0.103568,0.128955,0.015226,0.181216,0.551690,120.122558
std,39136.569008,5.85996,21.335577,0.140416,0.152745,3.615059,1.933482,0.497254,0.096159,0.173346,0.087771,0.140669,0.220864,26.967112
min,113000.000000,1998.00000,0.000000,0.129000,0.054900,0.000000,-20.514000,0.000000,0.023200,0.000019,0.000000,0.021500,0.038100,60.019000
25%,203580.000000,2004.00000,56.000000,0.581000,0.622000,2.000000,-6.490250,0.000000,0.039600,0.014000,0.000000,0.088100,0.386750,98.985750
50%,223279.500000,2010.00000,65.500000,0.676000,0.736000,6.000000,-5.285000,1.000000,0.059850,0.055700,0.000000,0.124000,0.557500,120.021500
75%,248133.000000,2015.00000,73.000000,0.764000,0.839000,8.000000,-4.167750,1.000000,0.129000,0.176250,0.000068,0.241000,0.730000,134.265500
max,484146.000000,2020.00000,89.000000,0.975000,0.999000,11.000000,-0.276000,1.000000,0.576000,0.976000,0.985000,0.853000,0.973000,210.851000


In [21]:
songs.head()

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop


In [22]:
y = songs.pop('popularity')
X = songs.drop(columns=['song', 'artist', 'key'])
y

0       77
1       79
2       66
3       78
4       65
        ..
1995    79
1996    78
1997    69
1998    75
1999    85
Name: popularity, Length: 2000, dtype: int64

In [23]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(songs)
numerical_columns.remove('key')

categorical_columns = categorical_columns_selector(songs)
categorical_columns.remove('artist')
categorical_columns.remove('song')

print(f"---- Numerical columns: ----\n{numerical_columns}")
print(f"\n---- Categorical columns ----\n{categorical_columns}")

---- Numerical columns: ----
['duration_ms', 'explicit', 'year', 'danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

---- Categorical columns ----
['genre']


In [24]:
categorical_preprocessor = Pipeline(steps=[("encoder", OneHotEncoder(handle_unknown="ignore")), ("imputer", SimpleImputer(strategy="median"))])
num_preprocessor = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])

In [25]:
preprocessor = ColumnTransformer([
  ('Variables Categóricas',categorical_preprocessor, categorical_columns),
  ('Variables Numéricas',num_preprocessor, numerical_columns)
], remainder="passthrough")

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,train_size=0.7, random_state=123)
print(X_train.columns)
# y_train

Index(['duration_ms', 'explicit', 'year', 'danceability', 'energy', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'genre'],
      dtype='object')


## Modelo 1

In [27]:
model = make_pipeline(
  preprocessor,
  MLPRegressor(hidden_layer_sizes=(40, 50),
    activation='tanh',
    solver='adam', 
    alpha=10, 
    batch_size='auto', 
    learning_rate='constant', 
    learning_rate_init=0.001, 
    max_iter=500,
    shuffle=True, 
    random_state=None,
    tol=1,
    verbose=False, 
    warm_start=False,
    momentum=0.9,
    nesterovs_momentum=True, 
    early_stopping=False,
    validation_fraction=0.1,
    beta_1=0.9, 
    beta_2=0.999,
    epsilon=1e-08, 
    n_iter_no_change=100
    )
)
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Variables Categóricas',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['genre']),
                                                 ('Variables Numéricas',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['duration_ms', 'explicit',
                                                   'year', 'danceability',
                                                   'energy', 'loudness', 'mode',
                                                   'speechiness',
                                                   'acousticness',
                                                   'instrumentalness',
                                                   'liveness', 'valence',
                                                   'tempo'])])),
                ('mlpregressor',
                 MLPRegressor(activation='tanh', alpha=10,
                              hidden_layer_sizes=(40, 50), max_iter=500,
                              n_iter_no_change=100, tol=1))])

In [28]:
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Variables Categóricas',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['genre']),
                                                 ('Variables Numéricas',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['duration_ms', 'explicit',
                                                   'year', 'danceability',
                                                   'energy', 'loudness', 'mode',
                                                   'speechiness',
                                                   'acousticness',
                                                   'instrumentalness',
                                                   'liveness', 'valence',
                                                   'tempo'])])),
                ('mlpregressor',
                 MLPRegressor(activation='tanh', alpha=10,
                              hidden_layer_sizes=(40, 50), max_iter=500,
                              n_iter_no_change=100, tol=1))])

In [29]:
pred1 = model.predict(X_test)

In [30]:
mse = mean_squared_error(y_test, pred1)
print(f"MSE (test): {round(mse, 3)}")
mae_test = mean_absolute_error(y_test, pred1)
print(f"MAE (test): {round(mae_test, 3)}")
rmse_test = np.sqrt(mse)
print(f"RMSE (test): {round(rmse_test, 3)}")

MSE (test): 457.424
MAE (test): 14.956
RMSE (test): 21.387


## Modelo 2

In [31]:
model1 = make_pipeline(
  preprocessor,
  MLPRegressor(hidden_layer_sizes=(100, 50),
    activation='logistic',
    solver='adam',
    alpha=0.001,
    batch_size='auto',
    learning_rate='constant',
    learning_rate_init=0.0002,
    max_iter=500,
    shuffle=True,
    random_state=None,
    tol=0.0001,
    verbose=True,
    warm_start=False,
    momentum=0.9,
    nesterovs_momentum=True,
    early_stopping=False,
    validation_fraction=0.1,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-20,
    n_iter_no_change=10
  )
)
model1

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Variables Categóricas',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['genre']),
                                                 ('Variables Numéricas',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['duration_ms', 'explicit',
                                                   'year', 'danceability',
                                                   'energy', 'loudness', 'mode',
                                                   'speechiness',
                                                   'acousticness',
                                                   'instrumentalness',
                                                   'liveness', 'valence',
                                                   'tempo'])])),
                ('mlpregressor',
                 MLPRegressor(activation='logistic', alpha=0.001, epsilon=1e-20,
                              hidden_layer_sizes=(100, 50),
                              learning_rate_init=0.0002, max_iter=500,
                              verbose=True))])

In [32]:
# Ajustar el modelo
model1.fit(X_train, y_train)

Iteration 1, loss = 2018.77612383
Iteration 2, loss = 2011.32662217
Iteration 3, loss = 2003.90840349
Iteration 4, loss = 1996.55690515
Iteration 5, loss = 1989.30491042
Iteration 6, loss = 1982.15603249
Iteration 7, loss = 1975.03449369
Iteration 8, loss = 1968.11103504
Iteration 9, loss = 1961.23747601
Iteration 10, loss = 1954.49834342
Iteration 11, loss = 1947.87005650
Iteration 12, loss = 1941.43488769
Iteration 13, loss = 1935.05128167
Iteration 14, loss = 1928.81922667
Iteration 15, loss = 1922.76114241
Iteration 16, loss = 1916.82691384
Iteration 17, loss = 1911.03146990
Iteration 18, loss = 1905.32856993
Iteration 19, loss = 1899.83946295
Iteration 20, loss = 1894.46293590
Iteration 21, loss = 1889.23964984
Iteration 22, loss = 1884.12286461
Iteration 23, loss = 1879.13296810
Iteration 24, loss = 1874.27723611
Iteration 25, loss = 1869.55685275
Iteration 26, loss = 1864.93328587
Iteration 27, loss = 1860.43324014
Iteration 28, loss = 1856.03865374
Iteration 29, loss = 1851.723

C:\Users\dellg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Variables Categóricas',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['genre']),
                                                 ('Variables Numéricas',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['duration_ms', 'explicit',
                                                   'year', 'danceability',
                                                   'energy', 'loudness', 'mode',
                                                   'speechiness',
                                                   'acousticness',
                                                   'instrumentalness',
                                                   'liveness', 'valence',
                                                   'tempo'])])),
                ('mlpregressor',
                 MLPRegressor(activation='logistic', alpha=0.001, epsilon=1e-20,
                              hidden_layer_sizes=(100, 50),
                              learning_rate_init=0.0002, max_iter=500,
                              verbose=True))])

In [33]:
pred2 = model1.predict(X_test)

In [34]:
mse = mean_squared_error(y_test, pred1)
print("MSE (test):", mse)
mae_test = mean_absolute_error(y_test, pred1)
print("MAE (test):", mae_test)
rmse_test = np.sqrt(mse)
print("RMSE (test):", rmse_test)

MSE (test): 457.42406428754134
MAE (test): 14.955921831075788
RMSE (test): 21.387474471931963
